# Reactions
This notebook contains all the steps followed in order to reconcile the existing metabolic reconstruction in CHO cells. It si divided into two parts: **1. Network Reconstruction** and **2. Identification of duplicated reactions**

[1. Network Reconstruction](#reconstruction) <br>
&nbsp;&nbsp;&nbsp;&nbsp;**1.1 Datasets generation and merge**<br>
&nbsp;&nbsp;&nbsp;&nbsp;**1.2 Normalization of the data** <br>
&nbsp;&nbsp;&nbsp;&nbsp;**1.3 Group all the data into a unified dataset** <br>
&nbsp;&nbsp;&nbsp;&nbsp;**1.4 Addition of the Recon3D CHO ortholog GPRs into the reconstruction** <br>
&nbsp;&nbsp;&nbsp;&nbsp;**1.5 Divide the dataset in two to facilitate reading and curation in Google Sheets** <br>

[2. Identification of Duplicated Reactions](#duplicated) <br>
&nbsp;&nbsp;&nbsp;&nbsp;**2.1 Retrieve information from the Google Sheet datasets reactions and metabolites**<br>
&nbsp;&nbsp;&nbsp;&nbsp;**2.2 Build a model and feed it the information from the df generated** <br>
&nbsp;&nbsp;&nbsp;&nbsp;**2.3 Save and validate the model** <br>

## 1. Network Reconstruction <a id='reconstruction'></a>
These include the generation of datasets (1), normalization of the data (2), merging all four reconstructions into a unified dataset (3), compiling the generated reconstruction into a cobra model in order to identify duplicated reactions (4), fixing duplicated reactions using BiGG ID annotation (5), adding Recon3D GPR information (6), and finally dividing the dataset into two different datasets that will be further curated in Google Sheets (7).

In [ ]:
# Libraries import
import pandas as pd
import numpy as np

### 1.1 Datasets generation and merge
Dataset generation from previous reconstructions (CHO_DG44, iCHO1766, iCHO2101, iCHO2291)

In [ ]:
#Read excel files and create the dfs

#Camels CHO_DG44 metabolic reconstruction
camel_df = pd.read_excel('../Data/Reconciliation/datasets/CHO_DG44.xlsx', header = 1)

#Hefzi's iCHO1766 metabolic reconstruction
hefzi_df = pd.read_excel('../Data/Reconciliation/datasets/hefzi_final.xlsx')

#Foudaliha's iCHO2101 metabolic reconstruction
fouladiha_df = pd.read_excel('../Data/Reconciliation/datasets/iCHO2101.xlsx', 'Supplementary Table 10', header = 1)

#Yeo's iCHO2291 metabolic reconstruction
iCHO2291 = pd.read_excel('../Data/Reconciliation/datasets/iCHO2291_final.xlsx')

### 1.2 Normalization of the data
All dataset are normalized into the same shape and format and then combined into one big dataset

In [ ]:
#Standarization of the columns names
camel_df.rename(columns = {'Reaction ID':'Reaction', 'Initial reaction in model':'Reaction Formula', 'Reaction name':'Reaction Name', 'Justification':'Curation Notes'}, inplace = True)
fouladiha_df.rename(columns = {'Abbreviation':'Reaction', 'Description':'Reaction Name', 'Reaction':'Reaction Formula', 'GPR':'GPR_fou'}, inplace = True)

# Addition of tag columns for organizational purposes
camel_df.insert(loc=0, column='cam', value='X')
camel_df.insert(loc=1, column='hef', value=np.nan)
camel_df.insert(loc=2, column='fou', value=np.nan)
camel_df.insert(loc=3, column='yeo', value=np.nan)

hefzi_df.insert(loc=0, column='cam', value=np.nan)
hefzi_df.insert(loc=1, column='hef', value='X')
hefzi_df.insert(loc=2, column='fou', value=np.nan)
hefzi_df.insert(loc=3, column='yeo', value=np.nan)

fouladiha_df.insert(loc=0, column='cam', value=np.nan)
fouladiha_df.insert(loc=1, column='hef', value=np.nan)
fouladiha_df.insert(loc=2, column='fou', value='X')
fouladiha_df.insert(loc=3, column='yeo', value=np.nan)

iCHO2291.insert(loc=0, column='cam', value=np.nan)
iCHO2291.insert(loc=1, column='hef', value=np.nan)
iCHO2291.insert(loc=2, column='fou', value=np.nan)
iCHO2291.insert(loc=3, column='yeo', value='X')

In [ ]:
# Generate cols list with the column names from all datasets
cols = hefzi_df.columns.to_list()+fouladiha_df.columns.to_list()+iCHO2291.columns.to_list()+camel_df.columns.to_list()

# Eliminate repetitive values in the 'cols' list
cols = [cols[i] for i in range(len(cols)) if i == cols.index(cols[i])]

In [ ]:
def add_col(df):
    '''
    This function adds the columns from the cols list 
    that are not present in the df
    '''
    df.columns
    add_col = []
    for col in cols:
        if col not in df.columns:
            add_col.append(col)
    df = df.reindex(columns = df.columns.tolist() + add_col)
    return df

In [ ]:
# Unify columns for all datasets
hefzi_df = add_col(hefzi_df)
fouladiha_df = add_col(fouladiha_df)
iCHO2291 = add_col(iCHO2291)
camel_df = add_col(camel_df)

In [ ]:
# Reorder columns in all datasets the same way
fouladiha_df = fouladiha_df[['cam','hef', 'fou', 'yeo', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR_hef','GPR_fou','GPR_yeo', 'Subsystem', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References', 'Reaction ID Camels Models']]
iCHO2291 = iCHO2291[['cam','hef', 'fou', 'yeo', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR_hef','GPR_fou','GPR_yeo', 'Subsystem', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References', 'Reaction ID Camels Models']]
hefzi_df = hefzi_df[['cam','hef', 'fou', 'yeo', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR_hef','GPR_fou','GPR_yeo', 'Subsystem', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References', 'Reaction ID Camels Models']]
camel_df = camel_df[['cam','hef', 'fou', 'yeo', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR_hef','GPR_fou','GPR_yeo', 'Subsystem', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References', 'Reaction ID Camels Models']]

camel_df['Reaction'] = camel_df['Reaction'].str.strip()

In [ ]:
# Merge all the dfs into a unified df
all_dfs = pd.concat([camel_df, hefzi_df, fouladiha_df, iCHO2291])
all_dfs = all_dfs.reset_index(drop = True)

#Unify reaction names
all_dfs['Reaction'] = all_dfs['Reaction'].str.replace('_cho', '')
all_dfs['Reaction'] = all_dfs['Reaction'].str.replace(r"(e)", "_e_", regex = False)
all_dfs['Reaction'] = all_dfs['Reaction'].str.replace("[", "_")
all_dfs['Reaction'] = all_dfs['Reaction'].str.replace("]", "_")

all_dfs #20940 rows/reactions (many of them repeated)

In [ ]:
# Remove the underscore at the end of some reactions

rxns = []
idx = []

for i,row in all_dfs.iterrows():
    if str(row['Reaction']).endswith('_'):
        s = re.sub('_$', '', row['Reaction'])
        rxns.append(s)
        idx.append(i)
        
all_dfs['Reaction'].update(pd.Series(rxns,index=idx))

### 1.3 Group all the data into a unified dataset
The combined dataset generated above "all_dfs" is grouped by the Reaction BiGG ID to obtain a dataset with unique reaction identifiers

In [ ]:
# Group the data into a unified dataset

all_dfs2 = all_dfs.groupby('Reaction').first()
all_dfs2['Reaction Formula'] = all_dfs2['Reaction Formula'].str.replace('[','_')
all_dfs2['Reaction Formula'] = all_dfs2['Reaction Formula'].str.replace(']','')
all_dfs2['Reaction Formula'] = all_dfs2['Reaction Formula'].str.replace(' => | =>',' --> ')
all_dfs2['Reaction Formula'] = all_dfs2['Reaction Formula'].str.replace(' <-- | <--',' <=> ')
all_dfs2

In [ ]:
# Separate Demand Reaccion into a different df and remove from reconstruction. 
# Keep demand reactions from extracellular space

all_dfs2 = all_dfs2.reset_index()
demand_reactions = pd.DataFrame(columns = all_dfs2.columns)

for index, rxn in all_dfs2.iterrows():
    rxn = rxn['Reaction Formula']
    a,b = re.split('<=>|-->',rxn)
    if (b == '' or b == ' ') and not a.endswith('_e '):
        temp_df = all_dfs2[all_dfs2['Reaction Formula'] == rxn]
        demand_reactions = pd.concat([demand_reactions,temp_df])
        all_dfs2.drop(index, inplace=True)

demand_reactions = demand_reactions.reset_index(drop=True)
demand_reactions.to_excel('../Data/Reconciliation/datasets/demand_reactions.xlsx')
demand_reactions

In [ ]:
all_dfs2 = all_dfs2.reset_index(drop=True)
all_dfs2

### 1.4 Addition of the Recon3D CHO ortholog GPRs into the reconstruction
The information from the dataset generated in the notebook "GPR Annotation" containing all the information from Recon3D GPRS in human and its corresponding CHO orthologs is mapped into our reconstruction

In [ ]:
# Generate a dict with recon3d reactions as keys and the CHO GPRs as values.
recon3d = pd.read_excel('../Data/GPR_Curation/recon3D_chogprs.xlsx')
recon3d_dict = recon3d.set_index('m_reaction')['CHO GPR'].to_dict()
recon3d_dict

In [ ]:
# Map 'recon3d_dict' into the all_dfs3 dataframe
# the reaction IDs should be the same as those in our reconstruction

all_dfs3 = all_dfs2.reset_index()
all_dfs3['GPR_Recon3D'] = all_dfs3['Reaction'].map(recon3d_dict)

### 1.5 Divide the dataset in two to facilitate reading and curation in Google Sheets
The dataset all_dfs3 is divided into two different datasets: all_dfs4 contains mainly all the information regarding GPRs assigned from previous reconstructions. all_dfs5 contains the rest of the attributes in the reconstruction such as EC number, bounds, etc.

In [ ]:
# all_dfs4 contains mainly information of the reactions GPRs
all_dfs4 = all_dfs3[['Reaction', 'Reaction Name', 'Reaction Formula', 'Subsystem', 'GPR_hef', 'GPR_fou', 'GPR_yeo', 'GPR_Recon3D', 'Curation Notes', 'References']]
all_dfs4.insert(8,'GPR_final', '')
all_dfs4.to_excel('../Data/Reconciliation/datasets/all_dfs4.xlsx')
all_dfs4

In [ ]:
# all_dfs5 contains information of the rest of the attributes in our reconstruction
all_dfs5 = all_dfs3[['Reaction', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible', 'Lower bound', 'Upper bound', 'Objective']]
all_dfs5.to_excel('../Data/Reconciliation/datasets/all_dfs5.xlsx')
all_dfs5

## 2. Identifications of Duplicated Reactions <a id='duplicated'></a>

In [4]:
# Libraries import
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from requests_html import HTMLSession
from tqdm.notebook import tqdm
from cobra import Model, Reaction, Metabolite

from utils import duplicated_reactions
from google_sheet import GoogleSheet

### 2.1 Compile all_df2 dataframe into a cobra model to identify duplicated reactions
Generation of a cobra model from the dataset containing the grouped reactions "all_dfs2". This model will be used to identify duplicated reactions from the stoichiometric matrix of the model.

In [6]:
# Generate the necessary datasets for the identification of the duplicated reactions
KEY_FILE_PATH = 'credentials.json'
SPREADSHEET_ID_v3 = '1MlBXeHIKw8k8fZyXm-sN__AHTRSunJxar_-bqvukZws'

# Initialize the GoogleSheet object
sheet = GoogleSheet(SPREADSHEET_ID_v3, KEY_FILE_PATH)

# Read data from the Google Sheet
sheet_rxns = 'Rxns'
sheet_attributes = 'Attributes'

reactions = sheet.read_google_sheet(sheet_rxns)
rxns_attributes = sheet.read_google_sheet(sheet_attributes)

In [3]:
model = Model("iCHOxxxx")
lr = []

for _, row in all_dfs2.iterrows():
    r = Reaction(row['Reaction'])
    lr.append(r)
    
model.add_reactions(lr)
model

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-24


NameError: name 'all_dfs2' is not defined

In [ ]:
# Add reaction formulas
for i,r in enumerate(tqdm(model.reactions)):
    r.build_reaction_from_string(all_dfs2['Reaction Formula'][i])
    print(r)

In [ ]:
#Preview of the model with all its reactions and metabolites
model

In [ ]:
duplicated_reactions = duplicated_reactions(model)

In [ ]:
# List of duplicated reactions
duplicated_reactions.index

### 5. Fix duplicated reactions in the dataset from the list duplicated_reactions obtained above
Duplicated reactions are itereated in a for loop and mapped in the original dataset. A request is made in the 
BiGG database http://bigg.ucsd.edu with each of the duplicated reactions. If any of the duplicated reactions is in BiGG, the other reaction automatically changes its name to the one located in Bigg. This way we unifiy the names of our reactions to those in BiGG.

In [ ]:
all_dfs2 = all_dfs2.reset_index()

i = 0
for rxn in tqdm(duplicated_reactions.index):
    
    session = HTMLSession()
    
    response=session.get('http://bigg.ucsd.edu/models/iCHOv1/reactions/'+rxn[0])
    if response.status_code == 200:
        all_dfs2['Reaction'][all_dfs2['Reaction'] == rxn[1]] = rxn[0]
        print(f'1 Reaction {rxn[1]} changed for {rxn[0]} in CHOv1')
        i += 1
    
    elif response.status_code != 200:
        response=session.get('http://bigg.ucsd.edu/universal/reactions/'+rxn[0]) 
        if response.status_code == 200:
            all_dfs2['Reaction'][all_dfs2['Reaction'] == rxn[1]] = rxn[0]
            print(f'2 Reaction {rxn[1]} changed for {rxn[0]} in Bigg')
            i += 1
            
        elif response.status_code != 200:
            response=session.get('http://bigg.ucsd.edu/models/iCHOv1/reactions/'+rxn[1])
            if response.status_code == 200:
                all_dfs2['Reaction'][all_dfs2['Reaction'] == rxn[0]] = rxn[1]
                print(f'3 Reaction {rxn[0]} changed for {rxn[1]} in CHOv1')
                i += 1
                
            elif response.status_code != 200:
                response=session.get('http://bigg.ucsd.edu/universal/reactions/'+rxn[1])
                if response.status_code == 200:
                    all_dfs2['Reaction'][all_dfs2['Reaction'] == rxn[0]] = rxn[1]
                    print(f'4 Reaction {rxn[0]} changed for {rxn[1]} in Bigg')
                    i += 1
                elif response.status_code != 200:
                    all_dfs2['Reaction'][all_dfs2['Reaction'] == rxn[1]] = rxn[0]
                    print(f'5 Reaction {rxn[0]} changed for {rxn[1]} not present in Bigg DB')
                    i += 1

print(len(duplicated_reactions))
print(i)

In [ ]:
# all_dfs3 a procesed dataset with unique reactions, a combination of the 4 previous reconstructions
all_dfs3 = all_dfs2.groupby('Reaction').first()
del all_dfs3['index']
all_dfs3.to_excel('../Data/Reconciliation/datasets/all_dfs3.xlsx')
all_dfs3

### 8. Identification and removal of duplicated reactions in the CHO Network + Recon 3D dataset

In [5]:
# Libraries import
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from requests_html import HTMLSession
from tqdm.notebook import tqdm
from cobra import Model, Reaction, Metabolite

from utils import duplicated_reactions
from google_sheet import GoogleSheet

In [7]:
##### ----- Create a model from the reactions sheet ----- #####
model = Model("iCHO")
lr = []
for _, row in reactions.iterrows():
    r = Reaction(row['Reaction'])
    lr.append(r)    
model.add_reactions(lr)
model

Name,iCHO
Memory address,15e0bac80
Number of metabolites,0
Number of reactions,10547
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,


In [ ]:
##### ----- Add information to each one of the reactions ----- #####
for i,r in enumerate(tqdm(model.reactions)):
    print(r.id)
    r.build_reaction_from_string(reactions['Reaction Formula'][i])
    r.name = reactions['Reaction Name'][i]
    r.subsystem = reactions['Subsystem'][i]
    r.lower_bound = float(rxns_attributes['Lower bound'][i])
    r.upper_bound = float(rxns_attributes['Upper bound'][i])

In [9]:
duplicated_reactions = duplicated_reactions(model)
duplicated_reactions

array([[ 158, 4455],
       [1745, 2263],
       [2141, 2143],
       [3164, 9094],
       [3269, 7616],
       [3305, 9208],
       [5198, 8822],
       [6548, 9122],
       [7802, 7803],
       [8917, 9365],
       [9021, 9537],
       [9027, 9541]])

In [10]:
i = 0
for rxn in tqdm(duplicated_reactions):
    
    session = HTMLSession()
    
    response=session.get('http://bigg.ucsd.edu/models/iCHOv1/reactions/'+reactions['Reaction'][rxn[0]])
    if response.status_code == 200:
        if reactions['Reaction'][rxn[0]] == rxns_attributes['Reaction'][rxn[0]] and reactions.loc[rxn[1], 'Reaction'] == rxns_attributes.loc[rxn[1], 'Reaction']:
            print(f'1 Reaction {reactions.iloc[rxn[1],1]} changed for {reactions.iloc[rxn[0],1]} present in CHOv1 model')
            reactions.loc[rxn[1], 'Reaction'] = reactions['Reaction'][rxn[0]]
            rxns_attributes.loc[rxn[1], 'Reaction'] = rxns_attributes['Reaction'][rxn[0]]
            i += 1
    
    elif response.status_code != 200:
        response=session.get('http://bigg.ucsd.edu/models/iCHOv1/reactions/'+reactions['Reaction'][rxn[1]])
        if response.status_code == 200:
            if reactions['Reaction'][rxn[0]] == rxns_attributes['Reaction'][rxn[0]] and reactions.loc[rxn[1], 'Reaction'] == rxns_attributes.loc[rxn[1], 'Reaction']:
                print(f'2 Reaction {reactions.iloc[rxn[0],1]} changed for {reactions.iloc[rxn[1],1]} present in CHOv1 model')
                reactions.loc[rxn[0], 'Reaction'] = reactions['Reaction'][rxn[1]]
                rxns_attributes.loc[rxn[0], 'Reaction'] = rxns_attributes['Reaction'][rxn[1]]
                i += 1
            
        elif response.status_code != 200:
            response=session.get('http://bigg.ucsd.edu/universal/reactions/'+reactions['Reaction'][rxn[0]])
            if response.status_code == 200:
                if reactions['Reaction'][rxn[0]] == rxns_attributes['Reaction'][rxn[0]] and reactions.loc[rxn[1], 'Reaction'] == rxns_attributes.loc[rxn[1], 'Reaction']:
                    print(f'3 Reaction {reactions.iloc[rxn[1],1]} changed for {reactions.iloc[rxn[0],1]} present BiGG database')
                    reactions.loc[rxn[1], 'Reaction'] = reactions['Reaction'][rxn[0]]
                    rxns_attributes.loc[rxn[1], 'Reaction'] = rxns_attributes['Reaction'][rxn[0]]
                    i += 1
                
            elif response.status_code != 200:
                response=session.get('http://bigg.ucsd.edu/universal/reactions/'+reactions['Reaction'][rxn[1]])
                if response.status_code == 200:
                    if reactions['Reaction'][rxn[0]] == rxns_attributes['Reaction'][rxn[0]] and reactions.loc[rxn[1], 'Reaction'] == rxns_attributes.loc[rxn[1], 'Reaction']:
                        print(f'4 Reaction {reactions.iloc[rxn[0],1]} changed for {reactions.iloc[rxn[1],1]} present BiGG database')
                        reactions.loc[rxn[0], 'Reaction'] = reactions['Reaction'][rxn[1]]
                        rxns_attributes.loc[rxn[0], 'Reaction'] = rxns_attributes['Reaction'][rxn[1]]
                        i += 1
                elif response.status_code != 200:
                    if reactions['Reaction'][rxn[0]] == rxns_attributes['Reaction'][rxn[0]] and reactions.loc[rxn[1], 'Reaction'] == rxns_attributes.loc[rxn[1], 'Reaction']:
                        print(f'5 Reaction {reactions.iloc[rxn[1],1]} changed for {reactions.iloc[rxn[0],1]} not present in Bigg DB')
                        reactions.loc[rxn[1], 'Reaction'] = reactions['Reaction'][rxn[0]]
                        rxns_attributes.loc[rxn[1], 'Reaction'] = rxns_attributes['Reaction'][rxn[0]]
                        i += 1

print(len(duplicated_reactions))
print(i)

  0%|          | 0/12 [00:00<?, ?it/s]

4 Reaction 3HAD240p changed for HMR_3063 present BiGG database
4 Reaction DECR101n6p changed for FAOXC102C101x present BiGG database
1 Reaction FA182ACPH changed for FA1821ACPH present in CHOv1 model
2 Reaction GTPDH changed for r0120 present in CHOv1 model
2 Reaction HACD181 changed for RE3243R present in CHOv1 model
2 Reaction HCARNS changed for r0465 present in CHOv1 model
4 Reaction HMR_9734 changed for TYRDHINDOX present BiGG database
2 Reaction PPADCOAL changed for r0220 present in CHOv1 model
1 Reaction RE3492C changed for RE3491C present in CHOv1 model
2 Reaction UPPDC2 changed for r0774 present in CHOv1 model
2 Reaction XOLEST182CEH changed for r1180 present in CHOv1 model
2 Reaction XOLEST204CEH changed for r1184 present in CHOv1 model
12
12


In [11]:
# Store the original column order
column_order_rxns = reactions.columns.tolist()
column_order_att = rxns_attributes.columns.tolist()

# Group by 'BiGG ID' and keep the first non-null value in each group, then reset the index
reactions = reactions.groupby('Reaction').first().reset_index()
rxns_attributes = rxns_attributes.groupby('Reaction').first().reset_index()

# Rearrange the columns to the original order
reactions = reactions[column_order_rxns]
rxns_attributes = rxns_attributes[column_order_att]

In [12]:
# Update the Google Sheet with the modified DataFrame
sheet.update_google_sheet(sheet_rxns, reactions)
sheet.update_google_sheet(sheet_attributes, rxns_attributes)
print("Google Sheet updated.")

Google Sheet updated.


In [13]:
# Check that Reactions IDs and formulas are equal in "Rxns" and "Attributes" sheets
reactions = sheet.read_google_sheet(sheet_rxns)
rxns_attributes = sheet.read_google_sheet(sheet_attributes)

rxnsIDseq = list(reactions['Reaction']) == list(rxns_attributes['Reaction'])
if rxnsIDseq:
    print('Reaction IDs in the Rxns and Attributes Sheets are equal\n')
else:
    rxns_sheet_ids = set(list(reactions['Reaction']))
    attr_sheet_ids = set(list(rxns_attributes['Reaction']))
    print(f'Reaction IDs that are in Rxns Sheet and not in Attributes Sheet {rxns_sheet_forms - attr_sheet_forms}\n')
    print(f'Reaction IDs that are in Attributes Sheet and not in Rxns Sheet {attr_sheet_forms - rxns_sheet_forms}\n')

rxnsforseq = list(reactions['Reaction Formula']) == list(rxns_attributes['Reaction Formula'])
if rxnsforseq:
    print('Reaction Formulas in the Rxns and Attributes Sheets are equal')
else:
    rxns_sheet_forms = set(list(reactions['Reaction Formula']))
    attr_sheet_forms = set(list(rxns_attributes['Reaction Formula']))
    print(f'Reaction formulas that are in Rxns Sheet and not in Attributes Sheet {rxns_sheet_forms - attr_sheet_forms}\n')
    print(f'Reaction formulas that are in Attributes Sheet and not in Rxns Sheet {attr_sheet_forms - rxns_sheet_forms}\n')

Reaction IDs in the Rxns and Attributes Sheets are equal

Reaction Formulas in the Rxns and Attributes Sheets are equal


## Divide into compartments

In [27]:
from cobra.io import load_json_model
import pandas as pd

# model = read_sbml_model('iCHOv3_CHO_05072023.xml')
model = load_json_model('iCHOv3_CHO_05072023.json')

In [32]:
subsystem_list = []
for r in model.reactions:
    subsystem_list.append(r.subsystem)
print("Number of unique subsystems:", len(set(subsystem_list)))

Number of unique subsystems: 137


In [30]:
subsystem_counts = pd.Series(subsystem_list).value_counts().reset_index()
subsystem_counts.columns = ['Subsystem', 'RXN Counts']
subsystem_counts[subsystem_counts['RXN Counts'] == 1]


,Subsystem,RXN Counts
129,AMINO ACID DERIVATIVE,1
130,ASCORBATE AND ALDARATE METABOLISM,1
131,VITAMIN K METABOLISM,1
132,NUCLEOTIDE SUGAR METABOLISM,1
133,PENTOSE AND GLUCURONATE INTERCONVERSIONS,1
134,PROTEIN ASSEMBLY,1
135,COENZYME B BIOSYNTHESIS,1
136,BIOMASS AND MAINTENANCE FUNCTIONS,1


In [19]:
# TODO -Remove blocked reactions from packages
#      -Define the high-importance reactions for biomass production
#      -Rename subsystems with very few reactions

CYP2R1: h_c + nadph_c + o2_c + vitd3_c --> 25hvitd3_c + h2o_c + nadp_c
